## COM361 &mdash; Introdução a Otimização &mdash; 2023, Prof. Amit ##

# Controle Ótimo de Epidemia #

#### Helena de Souza Douroucas (helenadouroucas@poli.ufrj.br),
#### Pedro Nunes Pereira (pedronunes@poli.ufrj.br),
#### Vinícius Quintanilha Porto Gomes (viniciusporto@poli.ufrj.br)

*****

### Índice

1. [Introdução](#1.-Introdução)
1. [Modelo Matemático](#2.-Modelo-Matemático)
1. [Solução](#3.-Solução)
1. [Resultados e Discussão](#4.-Resultados-e-discussão)
1. [Subseção Opcional](#4.A.-Acrescente-subseções-se-necessário)
1. [Conclusão](#5.-Conclusão)
1. [Referências bibliográficas](#6.-Referências-bibliográficas)

## 1. Introdução ##

A disseminação global da COVID-19, que ocorreu de 2020 a 2022, marcou um período crítico, registrando alarmantes 700 milhões de casos e 7 milhões de mortes em todo o mundo. Este cenário destacou a necessidade de aprimorar a compreensão de modelos epidemiológicos e a necessidade urgente de estabelecer estratégias eficazes para o controle de epidemias. Este estudo visa abordar o controle ótimo de uma epidemia, empregando técnicas de otimização incorporadas ao software Julia.

O modelo adotado para essa análise é o modelo SIR(Susceptíveis-Infectados-Recuperados), uma estrutura que categoriza a população em três compartimentos distintos: Suscetíveis (S), Infectados (I) e Recuperados (R). Os processos interativos nesses compartimentos são regidos por equações a diferenças, fundamentadas no modelo proposto por Gaff e colaboradores em 2009. Nesse modelo, outras variáveis importantes são a taxa de vacinação (u1) e a taxa de tratamento(u2) que consiste nas principais ações realizadas para controlar a doença. Além disso, diversos parâmetros são considerados no modelo Taxa de crescimento natural da população por dia (μ), taxa de transmissão da doença(β), taxa de recuperação natural dos infectados(γ) e taxa de mortalidade dos infectados (δ) que são fatores comuns em qualquer doença e podem variar com o tempo também dependendo dos métodos de controle e tratamento da doença, porém não foram considerados nesse modelo.

Nesse contexto, o objetivo fundamental consiste em identificar políticas de controle ótimas, concentradas na gestão eficaz de vacinas e tratamentos, com o propósito de reduzir de maneira substancial o número de indivíduos infectados. Desta forma, serão explorados diversos métodos de otimização, considerando casos particulares, como a existência de restrições orçamentárias para a aplicação de vacinas e tratamentos, que consistem no valor máximo disponível por uma população para tratar uma doença. Um segundo caso que abordaremos consiste em realizar controle em tempo real, onde as decisões de intervenção são tomadas com base nas condições atuais do sistema. Além disso, serão examinadas estratégias específicas, como o controle exclusivo por meio da vacinação, ou seja, o tratamento não será um método de controle, enriquecendo a análise abordada neste estudo.

A partir do desenvolvimento e aplicação do modelo, assim como nos casos particulares, apresentaremos diversas conclusões encontradas para a realização do controle ótimo de epidemias.


## 2. Modelo matemático ##

O modelo  divide a população em 3 grupos diferentes: sucetíveis (S), infectados (I) e recuperados (R).
O modelo usado por Gaff e Schaeffer (2009) considera o caso contínuo e define as derivadas de S, I e R. Nesse trabalho, usaremos um modelo de tempo discreto, com unidade de tempo de um dia.

$$S[k+1] = S[k] + \mu N[k] - \beta \frac{S[k]I[k]}{N[k]} - u_1[k]S[k] - \mu \frac{N[k]S[k]}{K}$$

$$I[k+1] = I[k] + \beta \frac{S[k]I[k]}{N[k]} - (\delta + \gamma + u_2[k])I[k] - \mu \frac{N[k]I[k]}{K}$$

$$R[k+1] = R[k]- u_1[k]S[k] + (\gamma + u_2[k])I[k] - \mu \frac{N[k]R[k]}{K}$$

Variáveis do Modelo:

S[k]: Número de indivíduos suscetíveis no dia k.

I[k]: Número de indivíduos infectados no dia k.

R[k]: Número de indivíduos recuperados no dia k.

N[k]: População total no dia k.

$u1[k]$: Taxa de vacinação no dia k.

$u2[k]$: Taxa de tratamento no dia k.

Parâmetros do Modelo:

$T_f$: Número total de dias considerados no modelo.

$\mu$: Taxa de crescimento natural da população por dia.

$\beta$: Taxa de transmissão da doença.

$\gamma$: Taxa de recuperação natural dos infectados.

$\delta$: Taxa de mortalidade dos infectados.

K: Capacidade de carga da população.

m: Parâmetro que influencia o peso da taxa de vacinação na função objetivo.
S0, I0, R0: Condições iniciais de suscetíveis, infectados e recuperados.

O termo $\mu N[k]$ se refere ao crescimento natural da população por dia, supondo que quem nasce é sucetível e é proporcional a população total.

O termo $\beta \frac{S[k]I[k]}{N[k]}$ se refere a infecção de sucetíveis, sendo proporcional ao número de sucetíveis e a fração da população $\frac{I[k]}{N[k]}$ que está infectada.

O termo $u_1[k]S[k]$ se refere à vacinação de sucetíveis, sendo proporcional ao número de sucetíveis. Os vacinados deixam de ser sucetíveis e entram na população de "Removidos", uma vez que não podem mais ser infectados.

O termo $\mu \frac{N[k]X[k]}{K}$ se refere às mortes não relacionadas com a doença, onde X é o grupo (sucetível, infectado ou removido). Ele é propocional ao número de pessoas no grupo e a fração $\frac{N[k]}{K}$, onde K é a capacidade de carga. A capacidade de carga é valor para qual a população tende para um cenário onde não há doença. É possível verificar isso escolhendo $K = N[k]$, e os termos $\mu \frac{N[k]X[k]}{K} = \mu X[k]$ e $\mu N[k]$ irão se cancelar.

O termo $(\delta + \gamma + u_2[k])I[k]$ se refere a 3 fenômenos diferentes, morte de infectados, recuperação natural de infectados e recuperação via tratamento dos infectados, respectivamente. As três parcelas são proporcionais ao número de infectados. Os infectados que morreram não entram para o grupo de removidos, diferentemente dos recuperados.

A função objetivo usada no caso contínuo é:

$$\int _0^{T_f} \left[B_1 I(t) + B_2 \left[\frac{R(t)}{K}\right]^m u_1^2(t) + B_3 u_2^2(t) \right] dt$$

Onde $B_1, B_2, B_3, m$ são parâmetros que vão alterar os pesos de cada termo e é preciso minimizar a função objetivo.

Levando para o caso discreto temos:

$$\sum _{k=1}^{T_f} \left[B_1 I[k] + B_2 \left[\frac{R[k]}{K}\right]^m u_1^2[k] + B_3 u_2^2[k] \right]$$

O termo $B_1 I[k]$ é proporcial ao número de infectados.

O termo $B_2 \left[\frac{R[k]}{K}\right]^m u_1^2[k]$ envolve a taxa de vacinação $u_1$, e um fator $\left[\frac{R[k]}{K}\right]^m$, que terá um peso maior quanto maior a porcentagem de removidos.

O termo $B_3 u_2^2[k]$ é propocional ao taxa de tratamento $u_2$ ao quadrado.


As restrições envolvendo R[k+1] são quadrática, já as restrições envolvendo S[k+1] e I[k+1] e a função objetivo são não lineares, portanto o modelo será não-linear.

O artigo adiciona mais duas restrições lineares, $u_1(t) \le {u_1}_{max} = 0.1$ e $u_2(t) \le {u_2}_{max} = 0.6$, equivalente a $u_1[k] < 0.1$ e $u_2[k] < 0.6$. Essa escolha é equivalente a limitar a vacinação a no máximo 10% da população sucetível por dia e o tratamento a no máxima 60% por dia.

Dependendo da epidemia e da população que está sendo analisada (uma populosa capital de estado ou uma pequena cidade do interior), esses limites precisarão ser alterados. Também é presuposto que a vacinação está disponível desde o início do periodo analisado, o que pode não ser o caso.

Colocando na forma padrão, temos:

$$
\begin{aligned}
\underset{S, I, R, N, u_1, u_2 \in \mathbb{R}^{T_f}}{\text{minimize}}\qquad& \sum _{k=1}^{T_f} \left[B_1 I[k] + B_2 \left[\frac{R[k]}{K}\right]^m u_1^2[k] + B_3 u_2^2[k] \right] \\
\text{sujeito a:}\qquad& f_i(x) \le 0 && i=1,\dots,m\\
& u_1[k] \ge 0 && k=1,\dots,T_f\\
& u_1[k] \le {u_1}_{max} && k=1,\dots,T_f\\
& u_2[k] \ge 0 && k=1,\dots,T_f\\
& u_2[k] \le {u_2}_{max} && k=1,\dots,T_f\\
& S[1] = S_0 &&\\
& I[1] = I_0 &&\\
& R[1] = R_0 &&\\
& S[k+1] = S[k] + \mu N[k] - \beta \frac{S[k]I[k]}{N[k]} - u_1[k]S[k] - \mu \frac{N[k]S[k]}{K} && k=1,\dots,T_f -1\\
& I[k+1] = I[k] + \beta \frac{S[k]I[k]}{N[k]} - (\delta + \gamma + u_2[k])I[k] - \mu \frac{N[k]I[k]}{K} && k=1,\dots,T_f -1\\
& R[k+1] = R[k]- u_1[k]S[k] + (\gamma + u_2[k])I[k] - \mu \frac{N[k]R[k]}{K} && k=1,\dots,T_f -1\\
& N[k] = S[k] + I[k] + R[k] && k=1,\dots,T_f\\
\end{aligned}
$$


Esse problema foi usado como base para todas as simulações feitas, a partir dele, foi introduzida a seguinte restrição orçamentária, onde $a$ é custo para vacinar uma pessoa e $b$ é o custo do tratamento de uma pessoa. $$\sum _{k=1}^{T_f}(au_1[k]S[k] + bu_2[k]I[k] ) \leq dinheiro\_max$$

Posteriormente, foi implementado um controle preditivo em que a simulação é repetida diariamente considerando um intervalo menor de tempo para decidir que ação tomar no dia, dessa forma, a cada nova simulação  o valor ótimo é calculado com base em novos dados

Então a simulação original e o modelo preditivo foram repetidos usando apenas a vacinação como forma de controle, e por fim, foi observado o comportamento dessas simulações com a presença de incertezas nos valores dos parametros, e de S, I e R

## 3. Solução ##

Nesta etapa iremos apresentar a solução desenvolvida

Podemos utilizar os parâmetros para testar a função objetivo proposta e o modelo

In [ ]:
using JuMP, Ipopt, Plots

m = Model(Ipopt.Optimizer)

# Parâmetros
Tf = 90
mu, beta, gamma, delta, K = (0.00004 , 0.5, 0.1, 0.001, 6500000)
B1, B2, B3 = (1, 1000, 1000)
u1max, u2max = (0.01, 0.1)
S0, I0, R0 = (6000000,50000,0)


#set_silent(m)

@variable(m, S[1:Tf] >= 0)   #Número de Suscetiveis
@variable(m, I[1:Tf] >= 0)   #Número de Infectados
@variable(m, R[1:Tf] >= 0)   #Número de "Removidos"
@variable(m, N[1:Tf] >= 0)   #População total
@variable(m, u1[1:Tf] >= 0)  #Uso de vacinação
@variable(m, u2[1:Tf] >= 0)  #Uso de tratamento
@variable(m, objetivo[1:Tf])

#Condições iniciais
@constraint(m, S[1] == S0)
@constraint(m, I[1] == I0)
@constraint(m, R[1] == R0)

S_SC = zeros(1,Tf)
I_SC = zeros(1,Tf)
R_SC = zeros(1,Tf)
N_SC = zeros(1,Tf)

S_SC[1] = S0
I_SC[1] = I0
R_SC[1] = R0
N_SC[1] = S_SC[1] + I_SC[1] + R_SC[1]

for k in 1:Tf-1
    S_SC[k+1] = S_SC[k] + mu*N_SC[k] - beta*S_SC[k]*I_SC[k]/N_SC[k] - mu*N_SC[k]*S_SC[k]/K
    I_SC[k+1] = I_SC[k] + beta*S_SC[k]*I_SC[k]/N_SC[k] - (gamma + delta)*I_SC[k] - mu*N_SC[k]*I_SC[k]/K
    R_SC[k+1] = R_SC[k] + (gamma)*I_SC[k] - mu*N_SC[k]*R_SC[k]/K
    N_SC[k+1] = S_SC[k+1] + I_SC[k+1] + R_SC[k+1]
end


for i in 1:Tf
    set_start_value.(S[i], S_SC[i])
    set_start_value.(I[i], I_SC[i])
    set_start_value.(R[i], R_SC[i])
    set_start_value.(N[i], N_SC[i])
end



for i in 1:Tf
    @constraint(m, N[i] == S[i] + I[i] + R[i])
    @constraint(m, u1[i] <= u1max)
    @constraint(m, u2[i] <= u2max)

    #Função objetivo através do truque do epígrafo
    @NLconstraint(m, objetivo[i] >= B1*I[i] + (B2*(R[i]/K)^2)*u1[i]^2 + B3*u2[i]^2)

end

for k in 1:Tf-1
    @NLconstraint(m, S[k+1] == S[k] + mu*N[k] - u1[k]*S[k] - beta*S[k]*I[k]/N[k] - mu*N[k]*S[k]/K)

    @NLconstraint(m, I[k+1] == I[k] + beta*S[k]*I[k]/N[k] - (gamma + delta + u2[k])*I[k] - mu*N[k]*I[k]/K)

    @constraint(m, R[k+1] == R[k] + u1[k]*S[k] + (gamma + u2[k])*I[k] - mu*N[k]*R[k]/K)
end

@objective(m, Min, sum(objetivo))

optimize!(m)

# Resultados
Suscetiveis = JuMP.value.(S)
Infectados = JuMP.value.(I)
Removidos = JuMP.value.(R)
Total = JuMP.value.(N)
println(JuMP.value.(u1))
println(JuMP.value.(u2))
println(JuMP.value.(I))
println("Soma dos infectados: ",sum(JuMP.value.(I)))

# Gráfico
plot([Suscetiveis,Infectados,Removidos,Total], title="Evolução da epidemia por dia", label=["Suscetiveis" "Infectados" "Removidos" "Total"], linewidth=1, ylims=(0, K*1.4))

SyntaxError: ignored

Solução ótima dessa simulação:

In [ ]:
#utilização de vacina e tratamento
plot([JuMP.value.(u1),JuMP.value.(u2)], title="Uso de vacinação e tratamentos", label=["vacinação" "Tratamento"], linewidth=1)

Podemos agora restringir um orçamento máximo para vacinação (u1) e tratamento (u2), sendo **a** o custo por vacina, **b** o custo de tratamento por pessoa e **dinheiro_max** o valor total orçado.

In [ ]:
using JuMP, Ipopt, Plots

m = Model(Ipopt.Optimizer)

# Parâmetros
Tf = 90
mu, beta, gamma, delta, K = (0.00004 , 0.5, 0.1, 0.001, 6500000)
B1, B2, B3 = (1, 1000, 1000)
u1max, u2max = (0.01, 0.1)
S0, I0, R0 = (6000000,50000,0)
a,b, dinheiro_max = (20,50,100000)

set_silent(m)

@variable(m, S[1:Tf] >= 0)   #Número de Suscetiveis
@variable(m, I[1:Tf] >= 0)   #Número de Infectados
@variable(m, R[1:Tf] >= 0)   #Número de "Removidos"
@variable(m, N[1:Tf] >= 0)   #População total
@variable(m, u1[1:Tf] >= 0)  #Uso de vacinação
@variable(m, u2[1:Tf] >= 0)  #Uso de tratamento
@variable(m, objetivo[1:Tf])

#Condições iniciais
@constraint(m, S[1] == S0)
@constraint(m, I[1] == I0)
@constraint(m, R[1] == R0)

S_SC = zeros(1,Tf)
I_SC = zeros(1,Tf)
R_SC = zeros(1,Tf)
N_SC = zeros(1,Tf)

S_SC[1] = S0
I_SC[1] = I0
R_SC[1] = R0
N_SC[1] = S_SC[1] + I_SC[1] + R_SC[1]

for k in 1:Tf-1
    S_SC[k+1] = S_SC[k] + mu*N_SC[k] - beta*S_SC[k]*I_SC[k]/N_SC[k] - mu*N_SC[k]*S_SC[k]/K
    I_SC[k+1] = I_SC[k] + beta*S_SC[k]*I_SC[k]/N_SC[k] - (gamma + delta)*I_SC[k] - mu*N_SC[k]*I_SC[k]/K
    R_SC[k+1] = R_SC[k] + (gamma)*I_SC[k] - mu*N_SC[k]*R_SC[k]/K
    N_SC[k+1] = S_SC[k+1] + I_SC[k+1] + R_SC[k+1]
end


for i in 1:Tf
    set_start_value.(S[i], S_SC[i])
    set_start_value.(I[i], I_SC[i])
    set_start_value.(R[i], R_SC[i])
    set_start_value.(N[i], N_SC[i])
end

for i in 1:Tf
    @constraint(m, N[i] == S[i] + I[i] + R[i])
    @constraint(m, u1[i] <= u1max)
    @constraint(m, u2[i] <= u2max)


    #Função objetivo através do truque do epígrafo
    @NLconstraint(m, objetivo[i] >= B1*I[i] + (B2*(R[i]/K)^2)*u1[i]^2 + B3*u2[i]^2)

end

@constraint(m,sum((a*u1[i]*S[i] + b*u2[i]*I[i]) for i in 1:Tf) <=dinheiro_max)

for k in 1:Tf-1
    @NLconstraint(m, S[k+1] == S[k] + mu*N[k] - u1[k]*S[k] - beta*S[k]*I[k]/N[k] - mu*N[k]*S[k]/K)

    @NLconstraint(m, I[k+1] == I[k] + beta*S[k]*I[k]/N[k] - (gamma + delta + u2[k])*I[k] - mu*N[k]*I[k]/K)

    @constraint(m, R[k+1] == R[k] + u1[k]*S[k] + (gamma + u2[k])*I[k] - mu*N[k]*R[k]/K)
end

@objective(m, Min, sum(objetivo))

optimize!(m)

# Resultados
Suscetiveis = JuMP.value.(S)
Infectados = JuMP.value.(I)
Removidos = JuMP.value.(R)
Total = JuMP.value.(N)
println(JuMP.value.(u1))
println(JuMP.value.(u2))
println(JuMP.value.(I))
println("Soma dos infectados: ",sum(JuMP.value.(I)))

# Gráfico
plot([Suscetiveis,Infectados,Removidos,Total], title="Evolução da epidemia por dia", label=["Suscetiveis" "Infectados" "Removidos" "Total"], linewidth=1, ylims=(0, K*1.4))

Solução ótima dessa simulação:

In [ ]:
#utilização de vacina e tratamento
plot([JuMP.value.(u1),JuMP.value.(u2)], title="Uso de vacinação e tratamentos", label=["vacinação" "Tratamento"], linewidth=1)

Em seguida, utilizamos um modelo preditivo em que a simulação é repetida diariamente considerando intervalos menores de tempo (nota-se que o valores de interesse dessa simulação são os que vão do dia 1 ao dia 70)

In [ ]:
using JuMP, Ipopt, Plots

# Parâmetros
Tft=90
Tf = 20
mu, beta, gamma, delta, K = (0.00004 , 0.5, 0.1, 0.001, 6500000)
B1, B2, B3 = (1, 1000, 1000)
u1max, u2max = (0.01, 0.1)
S0, I0, R0 = (6000000,50000,0)


S_SC = zeros(1,Tft)
I_SC = zeros(1,Tft)
R_SC = zeros(1,Tft)
N_SC = zeros(1,Tft)

S_SC[1] = S0
I_SC[1] = I0
R_SC[1] = R0
N_SC[1] = S_SC[1] + I_SC[1] + R_SC[1]

for k in 1:Tft-1
    S_SC[k+1] = S_SC[k] + mu*N_SC[k] - beta*S_SC[k]*I_SC[k]/N_SC[k] - mu*N_SC[k]*S_SC[k]/K
    I_SC[k+1] = I_SC[k] + beta*S_SC[k]*I_SC[k]/N_SC[k] - (gamma + delta)*I_SC[k] - mu*N_SC[k]*I_SC[k]/K
    R_SC[k+1] = R_SC[k] + (gamma)*I_SC[k] - mu*N_SC[k]*R_SC[k]/K
    N_SC[k+1] = S_SC[k+1] + I_SC[k+1] + R_SC[k+1]
end

Suscetiveis = zeros(90)
Infectados = zeros(90)
Removidos = zeros(90)
Total = zeros(90)
vac = zeros(90)
trat = zeros(90)


Suscetiveis[1] = S_SC[1]
Infectados[1] = I_SC[1]
Removidos[1] = R_SC[1]
Total[1] = N_SC[1]

for j in 1:Tft-Tf+1
    m = Model(Ipopt.Optimizer)
    set_silent(m)


    @variable(m, S[1:Tf] >= 0)   #Número de Suscetiveis
    @variable(m, I[1:Tf] >= 0)   #Número de Infectados
    @variable(m, R[1:Tf] >= 0)   #Número de "Removidos"
    @variable(m, N[1:Tf] >= 0)   #População total
    @variable(m, u1[1:Tf] >= 0)  #Uso de vacinação
    @variable(m, u2[1:Tf] >= 0)  #Uso de tratamento
    @variable(m, objetivo[1:Tf])

    #Condições iniciais
    @constraint(m, S[1] == Suscetiveis[j])
    @constraint(m, I[1] == Infectados[j])
    @constraint(m, R[1] == Removidos[j])



    for i in 1:Tf
        set_start_value.(S[i], S_SC[i+j-1])
        set_start_value.(I[i], I_SC[i+j-1])
        set_start_value.(R[i], R_SC[i+j-1])
        set_start_value.(N[i], N_SC[i+j-1])
    end


    for i in 1:Tf
        @constraint(m, N[i] == S[i] + I[i] + R[i])
        @constraint(m, u1[i] <= u1max)
        @constraint(m, u2[i] <= u2max)

        #Função objetivo através do truque do epígrafo
        @NLconstraint(m, objetivo[i] >= B1*I[i] + (B2*(R[i]/K)^2)*u1[i]^2 + B3*u2[i]^2)

    end

    for k in 1:Tf-1
        @NLconstraint(m, S[k+1] == S[k] + mu*N[k] - u1[k]*S[k] - beta*S[k]*I[k]/N[k] - mu*N[k]*S[k]/K)

        @NLconstraint(m, I[k+1] == I[k] + beta*S[k]*I[k]/N[k] - (gamma + delta + u2[k])*I[k] - mu*N[k]*I[k]/K)

        @constraint(m, R[k+1] == R[k] + u1[k]*S[k] + (gamma + u2[k])*I[k] - mu*N[k]*R[k]/K)
    end

    @objective(m, Min, sum(objetivo))

    optimize!(m)
    for i in 1:Tf
    Suscetiveis[i+j-1] =JuMP.value.(S)[i]
    Infectados[i+j-1] = JuMP.value.(I)[i]
    Removidos[i+j-1] = JuMP.value.(R)[i]
    Total[i+j-1] = JuMP.value.(N)[i]
    vac[i+j-1] = JuMP.value.(u1)[i]
    trat[i+j-1] = JuMP.value.(u2)[i]
    end
end
# Resultados


println("Soma dos infectados: ",sum(Infectados))

# Gráfico
plot([Suscetiveis,Infectados,Removidos,Total], title="Evolução da epidemia por dia", label=["Suscetiveis" "Infectados" "Removidos" "Total"], linewidth=1, ylims=(0, K*1.4))

SyntaxError: ignored

Solução ótima dessa simulação:

In [ ]:
#utilização de vacina e tratamentos
plot([vac,trat], title="Uso de vacinação e tratamentos", label=["vacinação" "Tratamento"], linewidth=1)

NameError: ignored

Caso exista apenas vacinação (ou seja, $u_2 = 0$), a complexidade do modelo e o poder de intervenção são reduzidos.
Nesse caso o objetivo passa a ser buscar um valor ótimo para $u_1$.
Uma outra característica dessa variação do modelo é que a taxa de letalidade é fixa em $\frac{\delta}{\delta + \gamma}$, portanto o número de mortos pela doença é proporcional ao número de infectados.

Então observamos o resultado da primeira simulação e da simulação com controle preditivo sem o uso do tratamento, apenas vacina

In [ ]:
using JuMP, Ipopt, Plots

m = Model(Ipopt.Optimizer)

# Parâmetros teste
#Tf = 20
#mu, beta, gamma, delta, K = (0.00004 , 0.5, 0.1, 0.1, 5000)
#B1, B2, B3 = (1, 1000, 1000)
#u1max, = (0.01)
#S0, I0, R0 = (4500,499,1)

# Parâmetros
Tf = 90
mu, beta, gamma, delta, K = (0.00004 , 0.5, 0.1, 0.001, 6500000)
B1, B2, B3 = (1, 1000, 1000)
u1max = (0.01)
S0, I0, R0 = (6000000,50000,0)


#set_silent(m)

@variable(m, S[1:Tf] >= 0)   #Número de Suscetiveis
@variable(m, I[1:Tf] >= 0)   #Número de Infectados
@variable(m, R[1:Tf] >= 0)   #Número de "Removidos"
@variable(m, N[1:Tf] >= 0)   #População total
@variable(m, u1[1:Tf] >= 0)  #Uso de vacinação
@variable(m, objetivo[1:Tf])

#Condições iniciais
@constraint(m, S[1] == S0)
@constraint(m, I[1] == I0)
@constraint(m, R[1] == R0)

S_SC = zeros(1,Tf)
I_SC = zeros(1,Tf)
R_SC = zeros(1,Tf)
N_SC = zeros(1,Tf)

S_SC[1] = S0
I_SC[1] = I0
R_SC[1] = R0
N_SC[1] = S_SC[1] + I_SC[1] + R_SC[1]

for k in 1:Tf-1
    S_SC[k+1] = S_SC[k] + mu*N_SC[k] - beta*S_SC[k]*I_SC[k]/N_SC[k] - mu*N_SC[k]*S_SC[k]/K
    I_SC[k+1] = I_SC[k] + beta*S_SC[k]*I_SC[k]/N_SC[k] - (gamma + delta)*I_SC[k] - mu*N_SC[k]*I_SC[k]/K
    R_SC[k+1] = R_SC[k] + (gamma)*I_SC[k] - mu*N_SC[k]*R_SC[k]/K
    N_SC[k+1] = S_SC[k+1] + I_SC[k+1] + R_SC[k+1]
end


for i in 1:Tf
    set_start_value.(S[i], S_SC[i])
    set_start_value.(I[i], I_SC[i])
    set_start_value.(R[i], R_SC[i])
    set_start_value.(N[i], N_SC[i])
end



for i in 1:Tf
    @constraint(m, N[i] == S[i] + I[i] + R[i])
    @constraint(m, u1[i] <= u1max)

    #Função objetivo através do truque do epígrafo
    @NLconstraint(m, objetivo[i] >= B1*I[i] + (B2*(R[i]/K)^2)*u1[i]^2)

end

for k in 1:Tf-1
    @NLconstraint(m, S[k+1] == S[k] + mu*N[k] - u1[k]*S[k] - beta*S[k]*I[k]/N[k] - mu*N[k]*S[k]/K)

    @NLconstraint(m, I[k+1] == I[k] + beta*S[k]*I[k]/N[k] - (gamma + delta)*I[k] - mu*N[k]*I[k]/K)

    @constraint(m, R[k+1] == R[k] + u1[k]*S[k] + (gamma)*I[k] - mu*N[k]*R[k]/K)
end

@objective(m, Min, sum(objetivo))

optimize!(m)

# Resultados
Suscetiveis = JuMP.value.(S)
Infectados = JuMP.value.(I)
Removidos = JuMP.value.(R)
Total = JuMP.value.(N)
println(JuMP.value.(u1))
println(JuMP.value.(I))

Solução ótima dessa simulação:

In [ ]:
# Gráfico
a = plot([Suscetiveis,Infectados,Removidos,Total], title="Evolução da epidemia por dia", label=["Suscetiveis" "Infectados" "Removidos" "Total"], linewidth=1, ylims=(0, K*1.4))
b = plot([JuMP.value.(u1), Suscetiveis./Total], title="Evolução da epidemia por dia", label=["Taxa de vacinação por dia" "Fração S/N"], linewidth=1, ylims=(0, 1))
display(a)
display(b)

In [ ]:
#controle preditivo com u2=0
using JuMP, Ipopt, Plots

m = Model(Ipopt.Optimizer)

# Parâmetros teste
#Tf = 20
#mu, beta, gamma, delta, K = (0.00004 , 0.5, 0.1, 0.1, 5000)
#B1, B2, B3 = (1, 1000, 1000)
#u1max, = (0.01)
#S0, I0, R0 = (4500,499,1)

# Parâmetros
Tft=90
Tf = 20
mu, beta, gamma, delta, K = (0.00004 , 0.5, 0.1, 0.001, 6500000)
B1, B2, B3 = (1, 1000, 1000)
u1max = (0.01)
S0, I0, R0 = (6000000,50000,0)

#set_silent(m)

S_SC = zeros(1,Tft)
I_SC = zeros(1,Tft)
R_SC = zeros(1,Tft)
N_SC = zeros(1,Tft)

S_SC[1] = S0
I_SC[1] = I0
R_SC[1] = R0
N_SC[1] = S_SC[1] + I_SC[1] + R_SC[1]

for k in 1:Tft-1
    S_SC[k+1] = S_SC[k] + mu*N_SC[k] - beta*S_SC[k]*I_SC[k]/N_SC[k] - mu*N_SC[k]*S_SC[k]/K
    I_SC[k+1] = I_SC[k] + beta*S_SC[k]*I_SC[k]/N_SC[k] - (gamma + delta)*I_SC[k] - mu*N_SC[k]*I_SC[k]/K
    R_SC[k+1] = R_SC[k] + (gamma)*I_SC[k] - mu*N_SC[k]*R_SC[k]/K
    N_SC[k+1] = S_SC[k+1] + I_SC[k+1] + R_SC[k+1]
end

Suscetiveis = zeros(90)
Infectados = zeros(90)
Removidos = zeros(90)
Total = zeros(90)
vac = zeros(90)
trat = zeros(90)


Suscetiveis[1] = S_SC[1]
Infectados[1] = I_SC[1]
Removidos[1] = R_SC[1]
Total[1] = N_SC[1]

for j in 1:Tft-Tf+1
    m = Model(Ipopt.Optimizer)
    set_silent(m)


    @variable(m, S[1:Tf] >= 0)   #Número de Suscetiveis
    @variable(m, I[1:Tf] >= 0)   #Número de Infectados
    @variable(m, R[1:Tf] >= 0)   #Número de "Removidos"
    @variable(m, N[1:Tf] >= 0)   #População total
    @variable(m, u1[1:Tf] >= 0)  #Uso de vacinação
    @variable(m, objetivo[1:Tf])

    #Condições iniciais
    @constraint(m, S[1] == Suscetiveis[j])
    @constraint(m, I[1] == Infectados[j])
    @constraint(m, R[1] == Removidos[j])



    for i in 1:Tf
        set_start_value.(S[i], S_SC[i+j-1])
        set_start_value.(I[i], I_SC[i+j-1])
        set_start_value.(R[i], R_SC[i+j-1])
        set_start_value.(N[i], N_SC[i+j-1])
    end


    for i in 1:Tf
        @constraint(m, N[i] == S[i] + I[i] + R[i])
        @constraint(m, u1[i] <= u1max)

        #Função objetivo através do truque do epígrafo
        @NLconstraint(m, objetivo[i] >= B1*I[i] + (B2*(R[i]/K)^2)*u1[i]^2)

    end

    for k in 1:Tf-1
        @NLconstraint(m, S[k+1] == S[k] + mu*N[k] - u1[k]*S[k] - beta*S[k]*I[k]/N[k] - mu*N[k]*S[k]/K)

        @NLconstraint(m, I[k+1] == I[k] + beta*S[k]*I[k]/N[k] - (gamma + delta)*I[k] - mu*N[k]*I[k]/K)

        @constraint(m, R[k+1] == R[k] + u1[k]*S[k] + (gamma)*I[k] - mu*N[k]*R[k]/K)
    end

    @objective(m, Min, sum(objetivo))

    optimize!(m)
    for i in 1:Tf
    Suscetiveis[i+j-1] =JuMP.value.(S)[i]
    Infectados[i+j-1] = JuMP.value.(I)[i]
    Removidos[i+j-1] = JuMP.value.(R)[i]
    Total[i+j-1] = JuMP.value.(N)[i]
    vac[i+j-1] = JuMP.value.(u1)[i]
    end
end

# Resultados
println(JuMP.value.(u1))
println(JuMP.value.(I))

Solução ótima dessa simulação:

In [ ]:
#gráfico de vacinas e tratamento do controle preditivo sem u2
a = plot([Suscetiveis,Infectados,Removidos,Total], title="Evolução da epidemia por dia", label=["Suscetiveis" "Infectados" "Removidos" "Total"], linewidth=1, ylims=(0, K*1.4))
b = plot([JuMP.value.(u1), Suscetiveis./Total], title="Evolução da epidemia por dia", label=["Taxa de vacinação por dia" "Fração S/N"], linewidth=1, ylims=(0, 1))
display(a)
display(b)

Simulação original na presença de incertezas:

Nessa simulação, os valores ótimos de $u_1$ e $u_2$ obtidos anteriormente são observados em um modelo com parametros com incertezas para avaliar seu desempenho

In [ ]:
using JuMP, Ipopt, Plots

m = Model(Ipopt.Optimizer)



Tf = 90
mu, beta, gamma, delta, K = (0.00004 , 0.5, 0.1, 0.001, 6500000)
B1, B2, B3 = (1, 1000, 1000)
u1max, u2max = (0.01, 0.1)
S0, I0, R0 = (6000000,50000,0)


#set_silent(m)

@variable(m, S[1:Tf] >= 0)   #Número de Suscetiveis
@variable(m, I[1:Tf] >= 0)   #Número de Infectados
@variable(m, R[1:Tf] >= 0)   #Número de "Removidos"
@variable(m, N[1:Tf] >= 0)   #População total
@variable(m, u1[1:Tf] >= 0)  #Uso de vacinação
@variable(m, u2[1:Tf] >= 0)  #Uso de tratamento
@variable(m, objetivo[1:Tf])

#Condições iniciais
@constraint(m, S[1] == S0)
@constraint(m, I[1] == I0)
@constraint(m, R[1] == R0)


#O caso sem controle foi usado para obter os valores iniciais do problema
S_SC = zeros(1,Tf)
I_SC = zeros(1,Tf)
R_SC = zeros(1,Tf)
N_SC = zeros(1,Tf)

S_SC[1] = S0
I_SC[1] = I0
R_SC[1] = R0
N_SC[1] = S_SC[1] + I_SC[1] + R_SC[1]
for k in 1:Tf-1
    S_SC[k+1] = S_SC[k] + mu*N_SC[k] - beta*S_SC[k]*I_SC[k]/N_SC[k] - mu*N_SC[k]*S_SC[k]/K
    I_SC[k+1] = I_SC[k] + beta*S_SC[k]*I_SC[k]/N_SC[k] - (gamma + delta)*I_SC[k] - mu*N_SC[k]*I_SC[k]/K
    R_SC[k+1] = R_SC[k] + (gamma)*I_SC[k] - mu*N_SC[k]*R_SC[k]/K
    N_SC[k+1] = S_SC[k+1] + I_SC[k+1] + R_SC[k+1]
end


for i in 1:Tf
    set_start_value.(S[i], S_SC[i])
    set_start_value.(I[i], I_SC[i])
    set_start_value.(R[i], R_SC[i])
    set_start_value.(N[i], N_SC[i])
end



for i in 1:Tf
    @constraint(m, N[i] == S[i] + I[i] + R[i])
    @constraint(m, u1[i] <= u1max)
    @constraint(m, u2[i] <= u2max)

    #Função objetivo através do truque do epígrafo
    @NLconstraint(m, objetivo[i] >= B1*I[i] + (B2*(R[i]/K)^2)*u1[i]^2 + B3*u2[i]^2)

end

for k in 1:Tf-1
    @NLconstraint(m, S[k+1] == S[k] + mu*N[k] - u1[k]*S[k] - beta*S[k]*I[k]/N[k] - mu*N[k]*S[k]/K)

    @NLconstraint(m, I[k+1] == I[k] + beta*S[k]*I[k]/N[k] - (gamma + delta + u2[k])*I[k] - mu*N[k]*I[k]/K)

    @constraint(m, R[k+1] == R[k] + u1[k]*S[k] + (gamma + u2[k])*I[k] - mu*N[k]*R[k]/K)
end

@objective(m, Min, sum(objetivo))

optimize!(m)

S_modelo1 = JuMP.value.(S)
I_modelo1 = JuMP.value.(I)
R_modelo1 = JuMP.value.(R)
N_modelo1 = JuMP.value.(N)
u1_modelo1 = JuMP.value.(u1)
u2_modelo1 = JuMP.value.(u2)
println(JuMP.value.(u2))
println(JuMP.value.(I))
println("Soma dos infectados: ",sum(JuMP.value.(I)))
println("Numero de dias: ",Tf)
plot([S_modelo1,I_modelo1,R_modelo1,N_modelo1], title="Evolução da epidemia por dia", label=["Suscetiveis" "Infectados" "Removidos" "Total"], linewidth=1, ylims=(0, K*1.4))

mu, beta, gamma, delta, K = (0.00004 , 0.8, 0.05, 0.05, 5000)
B1, B2, B3 = (1, 1000, 1000)
S0, I0, R0 = (4000,999,1)

S_modelo2 = zeros(1,Tf)
I_modelo2 = zeros(1,Tf)
R_modelo2 = zeros(1,Tf)
N_modelo2 = zeros(1,Tf)

S_modelo2[1] = S0
I_modelo2[1] = I0
R_modelo2[1] = R0
N_modelo2[1] = S_modelo2[1] + I_modelo2[1] + R_modelo2[1]

for k in 1:Tf-1
    S_modelo2[k+1] = S_modelo2[k] + mu*N_modelo2[k] - u1_modelo1[k]*S_modelo2[k] - beta*S_modelo2[k]*I_modelo2[k]/N_modelo2[k] - mu*N_modelo2[k]*S_modelo2[k]/K
    I_modelo2[k+1] = I_modelo2[k] + beta*S_modelo2[k]*I_modelo2[k]/N_modelo2[k] - (gamma + delta + u2_modelo1[k])*I_modelo2[k] - mu*N_modelo2[k]*I_modelo2[k]/K
    R_modelo2[k+1] = R_modelo2[k] + u1_modelo1[k]*S_modelo2[k]+ (gamma + u2_modelo1[k])*I_modelo2[k] - mu*N_modelo2[k]*R_modelo2[k]/K
    N_modelo2[k+1] = S_modelo2[k+1]  + I_modelo2[k+1] + R_modelo2[k+1]
end


println("Soma dos infectados: ",sum(JuMP.value.(I_modelo2)))
println(u1_modelo1)
println(I_modelo2)
println("Proporção suscetível: ", I_modelo1[10]/N_modelo1[10])
plot(1:Tf, [S_modelo2[1:Tf], I_modelo2[1:Tf], R_modelo2[1:Tf], N_modelo2[1:Tf]], title="Evolução da epidemia por dia", label=["Suscetiveis" "Infectados" "Removidos" "Total"], linewidth=1, ylims=(0, K*1.4))

Valores de $u_1$ e $u_2$ encontrados como valores ótimos e usados na simulação com incertezas

In [ ]:
#utilização de vacina e tratamento
plot([JuMP.value.(u1),JuMP.value.(u2)], title="Uso de vacinação e tratamentos", label=["vacinação" "Tratamento"], linewidth=1)

Comportamento do controle preditivo na presença de incertezas:

Esse modelo foi implementado com o objetivo de simular algo que pudesse ser implementado na realidade, sem a dependência de parametros precisos. Ele foi pensado de forma que diariamente a simulação é refeita usando o valor ideal dos parametros, o valor de $u_1$ e $u_2$ encontrado nessa simulação é usado para calcular os dados do dia seguinte com os valores reais dos parametros (valores com incerteza), em seguida é adicionada uma incerteza aos valores calculados de S, I e R e esses valores são usados para repetir a simulação no dia seguinte

In [ ]:
using JuMP, Ipopt, Plots



# Parâmetros
Tft=90
Tf = 20
mu, beta, gamma, delta, K = (0.00004 , 0.5, 0.1, 0.001, 6500000)
B1, B2, B3 = (1, 1000, 1000)
u1max, u2max = (0.01, 0.1)
S0, I0, R0 = (6000000,50000,0)

betar=0.5*(19*ones(90).+2*rand(90))/20
gammar=0.1*(19*ones(90).+2*rand(90))/20
mur=0.00004*(19*ones(90).+2*rand(90))/20
deltar=0.001*(19*ones(90).+2*rand(90))/20


S_SC = zeros(1,Tft)
I_SC = zeros(1,Tft)
R_SC = zeros(1,Tft)
N_SC = zeros(1,Tft)

S_SC[1] = S0
I_SC[1] = I0
R_SC[1] = R0
N_SC[1] = S_SC[1] + I_SC[1] + R_SC[1]

for k in 1:Tft-1
    S_SC[k+1] = S_SC[k] + mu*N_SC[k] - beta*S_SC[k]*I_SC[k]/N_SC[k] - mu*N_SC[k]*S_SC[k]/K
    I_SC[k+1] = I_SC[k] + beta*S_SC[k]*I_SC[k]/N_SC[k] - (gamma + delta)*I_SC[k] - mu*N_SC[k]*I_SC[k]/K
    R_SC[k+1] = R_SC[k] + (gamma)*I_SC[k] - mu*N_SC[k]*R_SC[k]/K
    N_SC[k+1] = S_SC[k+1] + I_SC[k+1] + R_SC[k+1]
end

Suscetiveis = zeros(90)
Infectados = zeros(90)
Removidos = zeros(90)
Total = zeros(90)
vacinas = zeros(90)
tratamento=zeros(90)



Suscetiveis[1] = S_SC[1]
Infectados[1] = I_SC[1]
Removidos[1] = R_SC[1]
Total[1] = N_SC[1]

for j in 1:Tft-Tf-1
    m = Model(Ipopt.Optimizer)
    set_silent(m)


    @variable(m, S[1:Tf] >= 0)   #Número de Suscetiveis
    @variable(m, I[1:Tf] >= 0)   #Número de Infectados
    @variable(m, R[1:Tf] >= 0)   #Número de "Removidos"
    @variable(m, N[1:Tf] >= 0)   #População total
    @variable(m, u1[1:Tf] >= 0)  #Uso de vacinação
    @variable(m, u2[1:Tf] >= 0)  #Uso de tratamento
    @variable(m, objetivo[1:Tf])

    #Condições iniciais
    @constraint(m, S[1] == Suscetiveis[j])
    @constraint(m, I[1] == Infectados[j])
    @constraint(m, R[1] == Removidos[j])




    for i in 1:Tf
        set_start_value.(S[i], S_SC[i+j-1])
        set_start_value.(I[i], I_SC[i+j-1])
        set_start_value.(R[i], R_SC[i+j-1])
        set_start_value.(N[i], N_SC[i+j-1])
    end



    for i in 1:Tf
        @constraint(m, N[i] == S[i] + I[i] + R[i])
        @constraint(m, u1[i] <= u1max)
        @constraint(m, u2[i] <= u2max)

        #Função objetivo através do truque do epígrafo
        @NLconstraint(m, objetivo[i] >= B1*I[i] + (B2*(R[i]/K)^2)*u1[i]^2 + B3*u2[i]^2)

    end

    for k in 1:Tf-1
        @NLconstraint(m, S[k+1] == S[k] + mu*N[k] - u1[k]*S[k] - beta*S[k]*I[k]/N[k] - mu*N[k]*S[k]/K)

        @NLconstraint(m, I[k+1] == I[k] + beta*S[k]*I[k]/N[k] - (gamma + delta + u2[k])*I[k] - mu*N[k]*I[k]/K)

        @constraint(m, R[k+1] == R[k] + u1[k]*S[k] + (gamma + u2[k])*I[k] - mu*N[k]*R[k]/K)
    end

    @objective(m, Min, sum(objetivo))

    optimize!(m)
    for i in 1:Tf
        Suscetiveis[i+j-1] =JuMP.value.(S)[i]
        Infectados[i+j-1] = JuMP.value.(I)[i]
        Removidos[i+j-1] = JuMP.value.(R)[i]
        Total[i+j-1] = JuMP.value.(N)[i]

    end
    vacinas[j]=JuMP.value.(u1)[1]
    tratamento[j]=JuMP.value.(u2)[1]

    Suscetiveis[j+1] = (Suscetiveis[j] + mur[j]*Total[j] - betar[j]*Suscetiveis[j]*Infectados[j]/Total[j] - mur[j]*Total[j]*Suscetiveis[j]/K)*(19+2*rand())/20
    Infectados[j+1] = (Infectados[j] + betar[j]*Suscetiveis[j]*Infectados[j]/Total[j] - (gammar[j] + deltar[j])*Infectados[j] - mur[j]*Total[j]*Infectados[j]/K)*(19+2*rand())/20
    Removidos[j+1] = (Removidos[j] + (gammar[j])*Infectados[j] - mur[j]*Total[j]*Removidos[j]/K)*(19+2*rand())/20
    Total[j+1] = (Suscetiveis[j+1] + Infectados[j+1] + Removidos[j+1])*(19+2*rand())/20

end
# Resultados


println("Soma dos infectados: ",sum(Infectados))

# Gráfico
plot([Suscetiveis,Infectados,Removidos,Total], title="Evolução da epidemia por dia", label=["Suscetiveis" "Infectados" "Removidos" "Total"], linewidth=1, ylims=(0, K*1.4))

Solução ótima dessa simulação:

In [ ]:
#utilização de vacina e tratamentos
plot([vacinas,tratamento], title="Uso de vacinação e tratamentos", label=["vacinação" "Tratamento"], linewidth=1)

## 4. Resultados e discussão ##

### Diferença entre o controle preditivo e a útilização de uma única simulação
A primeira simulação visa antecipar os eventos nos próximos 90 dias uma única vez, o que não é factível na realidade, uma vez que não podemos ter certeza do futuro durante uma pandemia. Esse fator explica a sua eficácia, evidenciada ao compará-la com o controle preditivo, que consegue antever apenas 20 dias à frente.
### Diferença entre as duas simulações na presença de incertezas
Observamos que o modelo preditivo oferece um resultado mais realista, visto que ele refaz a simulação diariamente diante de novos dados, enquanto o outro método simula a melhor forma de ação apenas uma vez, o que na vida real ficaria mais discrepante da realidade com o tempo
### Uso de restrição orçamentária
Outra conclusão possível foi observada ao atribuir um custo muito discrepante a u2 em comparação com u1 no modelo com restrição orçamentária. Nesse cenário, o modelo tende a favorecer o método de melhor custo, a ponto de, dependendo da discrepância, utilizar apenas uma forma de controle da epidemia. Além disso, em casos de orçamento reduzido, as estratégias de controle se esgotam rapidamente, transformando-se em um modelo sem controle. Por outro lado, se o orçamento for elevado, o número de infectados permanecerá baixo.






## 5. Conclusão ##

Em conclusão, a análise abrangente realizada neste estudo sobre o controle ótimo de epidemias forneceu conclusões importantes para a compreensão e gestão eficaz de surtos epidêmicos. A abordagem centrada no modelo SIR, baseado nas equações a diferenças propostas por Gaff e colaboradores em 2009, permitiu uma avaliação detalhada dos compartimentos de Suscetíveis (S), Infectados (I) e Recuperados (R), criando uma base sólida para estratégias de controle.

Ao explorar diversos métodos de otimização, incluindo casos específicos com restrições orçamentárias para vacinação e tratamentos, assim como a implementação de estratégias de controle em tempo real, nosso estudo concluiu que algumas maneiras de realizar o controle impactam mais que outras e que algumas maneiras são possíveis apenas na teoria. Plotar o gráfico comparando o uso da estratégia u1 contra u2 foi bastante útil para entender o impacto de cada uma das estratégias e o que acontece se utilizamos apenas uma estratégia ou limitamos o orçamento.

No futuro, este trabalho poderia ser aprimorado para contemplar outras formas de contenção de epidemias, além de realizar o controle também dos parâmetros, por meio, por exemplo, de lockdown, construção de hospital, disponibilização de máscaras, uso de produtos de limpeza e propaganda de métodos de proteção. Além disso, poderia incluir orçamentos mais complexos e logística de vacinação e tratamento, pois como a epidemia é um evento não esperado, muitos fatores precisam ser considerados durante sua reação de controle.

## 6. Referências bibliográficas ##

MUNDIAL, Worldometer. Estatísticas da COVID-19. Disponível em: https://www.worldometers.info/coronavirus/. Acesso em: 18/12/2023.


G1. População no Rio de Janeiro (RJ) é de 6.211.423 pessoas, aponta o Censo do IBGE. Disponível em: https://g1.globo.com/rj/rio-de-janeiro/noticia/2023/06/28/populacao-no-rio-de-janeiro-rj-e-de-6211423-pessoas-aponta-o-censo-do-ibge.ghtml. Acesso em: 18/12/2023.


GAFF, Holly; SCHAEFFER, Elsa. Optimal Control Applied to Vaccination and Treatment Strategies for Various Epidemiological Models. Mathematical Biosciences and Engineering, v. 6, n. 3, p. 269-292 Números de página, julho 2009.
